# Imports 

In [1]:
import numpy as np
import pandas as pd

D:\anaconda\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
D:\anaconda\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
D:\anaconda\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


# Get the data

In [2]:
%%time

CHOSEN_SHEETS_SUMMARY = ["England", "Scotland", "Wales", "Northern Ireland"]
CHOSEN_SHEETS_UNIS = ["Q01", "Q02", "Q03", "Q04", "Q05", "Q06", "Q07", "Q08", "Q09", "Q10", "Q11", "Q12", "Q13", "Q14", "Q15",
                     "Q16", "Q17", "Q18", "Q19", "Q20", "Q21", "Q22", "Q23", "Q24", "Q25", "Q26", "Q27"]
questions = ["Q01", "Q02", "Q03", "Q04", "Q05", "Q06", "Q07", "Q08", "Q09", "Q10", "Q11", "Q12", "Q13", "Q14", "Q15",
                     "Q16", "Q17", "Q18", "Q19", "Q20", "Q21", "Q22", "Q23", "Q24", "Q25", "Q26", "Q27"]
CHOSEN_SHEETS_COURSE = ["NSS1", "NSS2", "NSS3"]

summary_data_1819 = pd.read_excel(r"data/summary_data_2018_2019.xls", skiprows = 5, nrows = 37, sheet_name = CHOSEN_SHEETS_SUMMARY) #skip nhs placement questions
summary_data_2122 = pd.read_excel(r"data/summary_data_2021_2022.xls", skiprows = 5, nrows = 37, sheet_name = CHOSEN_SHEETS_SUMMARY) #skip nhs placement questions

unis_data_1819 = pd.read_excel(r"data/unis_2018_2019.xls", skiprows = 5, sheet_name = CHOSEN_SHEETS_UNIS, header = None)
unis_data_2122 = pd.read_excel(r"data/unis_2021_2022.xls", skiprows = 5, sheet_name = CHOSEN_SHEETS_UNIS, header = None)

course_data_18 = pd.read_excel(r"data/course_data_2018.xls", skiprows = 4, sheet_name = CHOSEN_SHEETS_COURSE, header = None)
course_data_19 = pd.read_excel(r"data/course_data_2019.xls", skiprows = 4, sheet_name = CHOSEN_SHEETS_COURSE, header = None)
course_data_21 = pd.read_excel(r"data/course_data_2021.xls", skiprows = 4, sheet_name = CHOSEN_SHEETS_COURSE, header = None)
course_data_22 = pd.read_excel(r"data/course_data_2022.xls", skiprows = 4, sheet_name = CHOSEN_SHEETS_COURSE, header = None)

Wall time: 49 s


# Cleaning

#### 1. Cleaning summary data

In [3]:
def weighted_avg(file, idx, left_side):
    if left_side:
        return (file.iloc[idx][1] * file.iloc[idx][2] + file.iloc[idx][3] * file.iloc[idx][4]) / (file.iloc[idx][2]+ file.iloc[idx][4])
    return (file.iloc[idx][5] * file.iloc[idx][6] + file.iloc[idx][7] * file.iloc[idx][8]) / (file.iloc[idx][6]+ file.iloc[idx][8])
    
def clean(file, sheet_name, left_side):
    #optimise so that first columns are dropped after usage so left_side not needed, and copy data so can be run multiple times
    df = file.get(sheet_name).copy()
    df.dropna(axis = "rows", inplace = True) #drop empty rows - just text
    df.reset_index(inplace = True, drop = True) #reset index

    average_pct = []
    response_counts = []
    for i in range(len(df)):
        average_pct.append(weighted_avg(df, i, left_side))
        if left_side:
            response_counts.append(df.iloc[i][2] + df.iloc[i][4])
        else:
            response_counts.append(df.iloc[i][6] + df.iloc[i][8]) 
        
    df.drop(df.columns[[1,2,3,4,5,6,7,8]], axis=1, inplace=True) #drop columns, no longer needed
    df["score"] = average_pct #add score as feature
    
    df = df.T #set transpose as we want Qs af features
    df.rename(columns= lambda x: "Q"+str(x+1), inplace = True) #rename
    df.drop("Unnamed: 0", inplace = True, axis = "rows") #drop question row, now in header
    df.rename(index = {"score": sheet_name}, inplace = True) #keep track of rows
    df["avg_res"] = sum(response_counts)/df.shape[1] #add respsonse count
    
    return df

In [4]:
df_18 = pd.DataFrame()
for sheet in CHOSEN_SHEETS_SUMMARY:
    temp_df = clean(summary_data_1819, sheet, True)
    df_18 = pd.concat([df_18, temp_df])
    
df_21 = pd.DataFrame()
for sheet in CHOSEN_SHEETS_SUMMARY:
    temp_df = clean(summary_data_2122, sheet, True)
    df_21 = pd.concat([df_21, temp_df])

df_19 = pd.DataFrame()
for sheet in CHOSEN_SHEETS_SUMMARY:
    temp_df = clean(summary_data_1819, sheet, False)
    df_19 = pd.concat([df_19, temp_df])
    
df_22 = pd.DataFrame()
for sheet in CHOSEN_SHEETS_SUMMARY:
    temp_df = clean(summary_data_2122, sheet, False)
    df_22 = pd.concat([df_22, temp_df])
    
#take average of two years
summary_1819 = (df_18 + df_19) / 2
summary_2122 = (df_21 + df_22) / 2

#add total row by getting weighted avg and sum of repsonses
values_1819 = []
for i in range(summary_1819.shape[1]-1):
    values_1819.append(sum(summary_1819.iloc[:,i] * summary_1819.iloc[:,-1]) / sum(summary_1819.iloc[:,-1]))
values_1819.append(sum(summary_1819.iloc[:,-1]))

values_2122 = []
for i in range(summary_2122.shape[1]-1):
    values_2122.append(sum(summary_2122.iloc[:,i] * summary_2122.iloc[:,-1]) / sum(summary_2122.iloc[:,-1]))
values_2122.append(sum(summary_2122.iloc[:,-1]))

summary_1819.loc["UK"] = np.array(values_1819)
summary_2122.loc["UK"] = np.array(values_2122)

summary_1819 = summary_1819.infer_objects() #make sure that values are stored as floats
summary_2122 = summary_2122.infer_objects()

#2122 has extra questions, not needed
summary_2122.drop(["Q28", "Q29", "Q30", "Q31", "Q32", "Q33", "Q34", "Q35", "Q36", "Q37"], axis = "columns", inplace = True)

#### 2. Cleaning unis data

In [5]:
def clean_unis(file, sheet_name):
    df = file.get(sheet_name).copy()
    df.drop([0,3,4,5,6,7,9,10,11,12,13,14], inplace = True, axis = "columns")
    df.dropna(inplace = True, axis = "rows")
    df.reset_index(inplace = True, drop = True) #reset row index
    df[sheet_name] = (df.iloc[:,1] + df.iloc[:,2]) / 2 #add average result of the two years
    df.rename(columns = {1:'University'}, inplace = True) #rename column
    df.drop([2,8], inplace = True, axis = "columns")
    
    return df

In [6]:
for sheet in CHOSEN_SHEETS_UNIS:
    #create df for q1, keep adding scores only afterwards, uni names are returned for clarity
    if sheet == "Q01":
        unis_1819 = clean_unis(unis_data_1819, sheet)
    else:
        unis_1819[sheet] = clean_unis(unis_data_1819, sheet).iloc[:,1]
        
for sheet in CHOSEN_SHEETS_UNIS:
    #create df for q1, keep adding scores only afterwards, uni names are returned for clarity
    if sheet == "Q01":
        unis_2122 = clean_unis(unis_data_2122, sheet)
    else:
        unis_2122[sheet] = clean_unis(unis_data_2122, sheet).iloc[:,1]
        
#only keep unis which are in all 4 years
u1819 = set(unis_1819["University"])
u2122 = set(unis_2122["University"])
intersec = u1819.intersection(u2122) #get intersections

#remove rows which are not in intersection
for index, row in unis_1819.iterrows():
    if row.University not in intersec:
        unis_1819.drop(index, inplace = True)
        
for index, row in unis_2122.iterrows():
    if row.University not in intersec:
        unis_2122.drop(index, inplace = True)
    
unis_1819.reset_index(inplace = True, drop = True) #reset row index
unis_2122.reset_index(inplace = True, drop = True) #reset row index

unis_1819 = unis_1819.infer_objects() #make sure that values are stored as floats
unis_2122 = unis_2122.infer_objects()

#### 3. Cleaning course data

In [7]:
def clean_courses(file, sheet, question_numbers, new_data):
    df = file.get(sheet).copy()
    #21-22 files do not include last column
    if new_data:
        df.drop([0,1,2,4,6,7,8,9,10,11,12,14,15,16], inplace = True, axis = "columns") #drop not needed stuff
    else:
        df.drop([0,1,2,4,6,7,8,9,10,11,12,14,15,16,17], inplace = True, axis = "columns") #drop not needed stuff
    df.rename(columns = {3: "course", 5:"question", 13: "result"}, inplace = True) #rename so I actually know what is what
    
    #setup df
    course_data = pd.DataFrame()
    course_data["course"] = df.groupby(by = ["course"], as_index = False).mean(numeric_only = True)["course"]
    
    #get average by question and course
    for q in question_numbers:
        temp = df[df["question"] == q].copy()
        course_data[q] = temp.groupby(by = ["course"], as_index = False).mean(numeric_only = True)["result"]
        
    return course_data

In [8]:
course_18 = clean_courses(course_data_18, CHOSEN_SHEETS_COURSE[0], CHOSEN_SHEETS_UNIS, False)
course_19 = clean_courses(course_data_19, CHOSEN_SHEETS_COURSE[0], CHOSEN_SHEETS_UNIS, False)
course_21 = clean_courses(course_data_21, CHOSEN_SHEETS_COURSE[0], CHOSEN_SHEETS_UNIS, True)
course_22 = clean_courses(course_data_22, CHOSEN_SHEETS_COURSE[0], CHOSEN_SHEETS_UNIS, True)

In [9]:
#save media
media_18 = course_18.iloc[5]
media_19 = course_19.iloc[14]
media_21 = course_21.iloc[14]
media_22 = course_22.iloc[14]

#remove Humanities and liberal arts (non-specific), doesnt exist in other years
#also remove comms and media because it needs to be moved terrible code, but it is late
course_18.drop([5,12], axis = "rows", inplace = True) 
course_18.reset_index(drop = True, inplace = True)
course_19.drop([14], axis = "rows", inplace = True) 
course_19.reset_index(drop = True, inplace = True)
course_21.drop([14], axis = "rows", inplace = True) 
course_21.reset_index(drop = True, inplace = True)
course_22.drop([14], axis = "rows", inplace = True) 
course_22.reset_index(drop = True, inplace = True)

#readding media and comms
course_18 = course_18.append(media_18, ignore_index = True)
course_19 = course_19.append(media_19, ignore_index = True)
course_21 = course_21.append(media_21, ignore_index = True)
course_22 = course_22.append(media_22, ignore_index = True)

C:\Users\Áron\AppData\Local\Temp\ipykernel_2764\2837721766.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  course_18 = course_18.append(media_18, ignore_index = True)
C:\Users\Áron\AppData\Local\Temp\ipykernel_2764\2837721766.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  course_19 = course_19.append(media_19, ignore_index = True)
C:\Users\Áron\AppData\Local\Temp\ipykernel_2764\2837721766.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  course_21 = course_21.append(media_21, ignore_index = True)
C:\Users\Áron\AppData\Local\Temp\ipykernel_2764\2837721766.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  course_22 = cour

In [10]:
course_1819 = pd.DataFrame()
course_1819 = course_19.copy()
course_1819.iloc[:,1:] = (course_18.iloc[:,1:] + course_19.iloc[:,1:])/2

course_2122 = pd.DataFrame()
course_2122 = course_21.copy()
course_2122.iloc[:,1:] = (course_21.iloc[:,1:] + course_22.iloc[:,1:])/2

course_1819 = course_1819.infer_objects() #make sure that values are stored as floats
course_2122 = course_2122.infer_objects()

#### 4. Cleaning both course and uni

In [11]:
def clean_courses_and_unis(file, sheet, question_numbers, new_data):
    df = file.get(sheet).copy()
    #21-22 files do not include last column
    if new_data:
        df.drop([0,2,4,6,7,8,9,10,11,12,14,15,16], inplace = True, axis = "columns") #drop not needed stuff
    else:
        df.drop([0,2,4,6,7,8,9,10,11,12,14,15,16,17], inplace = True, axis = "columns") #drop not needed stuff
    df.rename(columns = {1: "uni", 3: "course", 5:"question", 13: "result"}, inplace = True) #rename so I actually know what is what

    #make questions into feature grouped by uni and course
    df = pd.pivot_table(df, values = 'result', index=['uni','course'], columns ="question").reset_index()

    #some files weirdly record scale or NHS as well
    if "Scale01" in df.columns:
        df.drop(["Scale01","Scale02","Scale03","Scale04","Scale05","Scale06","Scale07","Scale08"], 
            inplace = True, axis = "columns") #drop not needed stuff
    if "ScaleNHS" in df.columns:
        df.drop(["ScaleNHS"], inplace = True, axis = "columns")
    if "NHS1" in df.columns:
        df.drop(["NHS1", "NHS2", "NHS3", "NHS4", "NHS5", "NHS6"], inplace = True, axis = "columns")
    
    return df

In [12]:
course_and_uni_18 = clean_courses_and_unis(course_data_18, CHOSEN_SHEETS_COURSE[0], CHOSEN_SHEETS_UNIS, False)
course_and_uni_19 = clean_courses_and_unis(course_data_19, CHOSEN_SHEETS_COURSE[0], CHOSEN_SHEETS_UNIS, False)
course_and_uni_21 = clean_courses_and_unis(course_data_21, CHOSEN_SHEETS_COURSE[0], CHOSEN_SHEETS_UNIS, True)
course_and_uni_22 = clean_courses_and_unis(course_data_22, CHOSEN_SHEETS_COURSE[0], CHOSEN_SHEETS_UNIS, True)

In [13]:
#rename 3 courses for correctness in 2018
art_idxs = course_and_uni_18[course_and_uni_18["course"] == 'Creative arts and design']["course"].index
course_and_uni_18.iloc[art_idxs, 1] = 'Design, and creative and performing arts'

geography_idxs = course_and_uni_18[course_and_uni_18["course"] ==  'Geographical and environmental studies']["course"].index
course_and_uni_18.iloc[geography_idxs, 1] = 'Geography, earth and environmental studies'

comm_idxs = course_and_uni_18[course_and_uni_18["course"] ==  'Communications and media']["course"].index
course_and_uni_18.iloc[comm_idxs, 1] = 'Media, journalism and communications'

#drop 1
course_and_uni_18 = course_and_uni_18[course_and_uni_18["course"] != 'Humanities and liberal arts (non-specific)']

In [14]:
#todo: remove unis that are not in all 4 years
u18 = set(course_and_uni_18.uni + course_and_uni_18.course)
u19 = set(course_and_uni_19.uni + course_and_uni_19.course)
u1819 = u18.intersection(u19)

u21 = set(course_and_uni_21.uni + course_and_uni_21.course)
u22 = set(course_and_uni_22.uni + course_and_uni_22.course)
u2122 = u21.intersection(u22)

In [15]:
#remove rows which are not in intersection
for index, row in course_and_uni_18.iterrows():
    if row.uni+row.course not in u1819:
        course_and_uni_18.drop(index, inplace = True)
course_and_uni_18.reset_index(drop = True, inplace = True)
        
#remove rows which are not in intersection
for index, row in course_and_uni_19.iterrows():
    if row.uni+row.course not in u1819:
        course_and_uni_19.drop(index, inplace = True)   
course_and_uni_19.reset_index(drop = True, inplace = True)

#remove rows which are not in intersection
for index, row in course_and_uni_21.iterrows():
    if row.uni+row.course not in u2122:
        course_and_uni_21.drop(index, inplace = True)
course_and_uni_21.reset_index(drop = True, inplace = True)
        
#remove rows which are not in intersection
for index, row in course_and_uni_22.iterrows():
    if row.uni+row.course not in u2122:
        course_and_uni_22.drop(index, inplace = True)
course_and_uni_22.reset_index(drop = True, inplace = True)

#combine two years
course_and_uni_1819 = pd.DataFrame()
course_and_uni_1819["uni"] = course_and_uni_18.uni
course_and_uni_1819["course"] = course_and_uni_18.course

#element wise mean
for q in questions:
    d18 = course_and_uni_18[q]
    d19 = course_and_uni_19[q]
    course_and_uni_1819[q] = np.mean(np.stack((course_and_uni_18[q], course_and_uni_19[q])), axis = 0)
    
#do it again
course_and_uni_2122 = pd.DataFrame()
course_and_uni_2122["uni"] = course_and_uni_21.uni
course_and_uni_2122["course"] = course_and_uni_21.course

#element wise mean
for q in questions:
    d21 = course_and_uni_21[q]
    d22 = course_and_uni_22[q]
    course_and_uni_2122[q] = np.mean(np.stack((course_and_uni_21[q], course_and_uni_22[q])), axis = 0)
    
course_and_uni_1819.dropna(inplace = True, axis = "rows")
course_and_uni_2122.dropna(inplace = True, axis = "rows")

# Save and load data

In [18]:
summary_1819.to_pickle("data/summary_1819.pkl")
summary_2122.to_pickle("data/summary_2122.pkl")
unis_1819.to_pickle("data/unis_1819.pkl")
unis_2122.to_pickle("data/unis_2122.pkl")
course_1819.to_pickle("data/course_1819.pkl")
course_2122.to_pickle("data/course_2122.pkl")
course_and_uni_1819.to_pickle("data/course_and_uni_1819.pkl")
course_and_uni_2122.to_pickle("data/course_and_uni_2122.pkl")

In [19]:
pd.read_pickle("data/summary_1819.pkl")

,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,...,Q19,Q20,Q21,Q22,Q23,Q24,Q25,Q26,Q27,avg_res
England,88.778160,81.924411,84.800519,81.225215,84.075056,84.886815,80.889026,72.525879,72.712395,74.600840,...,86.632812,86.817193,68.538242,83.659492,83.902753,75.447773,60.915480,55.902584,83.150482,274799.296296
Scotland,89.457671,83.012992,85.541630,80.554780,83.527915,84.040667,78.568853,71.681497,73.167350,65.166609,...,87.430486,87.890169,67.254589,85.835967,86.415676,74.458033,55.403336,52.364461,83.527527,24350.777778
Wales,89.935977,83.503667,84.982081,82.014255,84.407333,85.421609,82.467954,75.067815,75.181148,75.042689,...,86.807669,87.224629,71.852159,85.437866,86.321741,78.964212,64.790899,58.695526,84.920995,16079.574074
Northern Ireland,89.024086,82.205276,85.253734,83.277861,83.700776,85.201340,84.200163,73.878637,74.954569,68.803944,...,88.126640,88.126266,68.900289,88.274402,83.822866,74.552509,59.371378,57.498343,84.522136,7774.462963
UK,88.892944,82.091855,84.876337,81.263357,84.041340,84.857219,80.872410,72.621323,72.923558,73.772077,...,86.737608,86.949874,68.615155,84.023180,84.210696,75.526663,60.655686,55.813295,83.300060,323004.111111


In [20]:
pd.read_pickle("data/summary_2122.pkl")

,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,...,Q19,Q20,Q21,Q22,Q23,Q24,Q25,Q26,Q27,avg_res
England,79.737075,84.147697,77.662464,81.305213,75.903553,79.071665,79.465628,81.224499,76.590480,68.786830,...,69.585787,64.257502,74.258811,70.317157,77.127128,74.362189,79.472519,77.918467,67.198019,277392.405405
Scotland,82.109712,86.462607,80.596142,83.842190,77.596963,79.973417,81.229188,82.608500,76.144778,65.740663,...,71.009157,63.585140,78.076969,71.494098,78.667830,76.338623,80.074280,80.075420,67.866578,25740.797297
Wales,80.525789,85.011885,79.146727,81.539570,76.456929,79.132272,79.015333,81.310633,77.152465,69.522817,...,69.143669,62.646219,75.261070,69.634658,75.573563,73.152011,76.846685,77.104562,68.073062,15153.256757
Northern Ireland,81.095008,85.277551,77.698817,82.520124,78.952124,81.038071,80.047217,82.278878,80.933878,67.921884,...,75.258821,69.313610,80.855696,75.713174,79.762611,78.479675,80.273279,81.204810,70.509446,7575.351351
UK,79.992741,84.397011,77.964070,81.544757,76.133924,79.191428,79.597518,81.362342,76.682378,68.560322,...,69.809545,64.247002,74.760384,70.503831,77.237856,74.557758,79.416562,78.127401,67.368503,325861.810811


In [21]:
pd.read_pickle("data/unis_1819.pkl")

,University,Q01,Q02,Q03,Q04,Q05,Q06,Q07,Q08,Q09,...,Q18,Q19,Q20,Q21,Q22,Q23,Q24,Q25,Q26,Q27
0,University of Aberdeen,91.345,84.160,88.010,80.060,85.455,83.625,76.190,72.820,77.140,...,84.495,90.445,90.810,72.345,87.335,90.275,78.440,58.760,44.100,86.645
1,Abertay University,90.895,84.025,82.300,80.950,83.200,85.370,78.330,77.285,76.705,...,87.470,88.060,89.715,68.865,86.465,88.630,76.885,59.520,53.505,82.965
2,Aberystwyth University,95.120,89.995,90.100,85.585,89.530,89.910,86.170,84.175,82.035,...,91.680,91.530,91.270,75.230,88.050,89.925,85.540,77.150,58.170,90.470
3,Abingdon and Witney College,94.310,92.630,91.380,87.960,84.600,88.250,82.260,78.610,82.040,...,75.910,75.335,83.585,71.545,89.340,86.545,87.170,72.340,47.770,89.215
4,ACM Guildford Limited,80.725,68.250,51.025,50.355,65.165,65.820,65.425,52.195,60.040,...,61.215,53.395,64.325,59.545,75.470,65.185,61.785,42.940,36.845,50.085
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
348,University of York,92.190,86.290,88.200,78.945,86.855,85.240,78.855,73.815,75.710,...,88.830,88.955,90.175,67.375,84.395,88.740,76.945,58.905,42.825,86.635
349,York College,88.030,82.215,77.800,77.185,73.715,76.060,78.795,71.130,79.185,...,67.935,74.605,70.590,65.855,76.210,72.150,74.320,52.810,36.535,75.000
350,York St John University,91.440,85.700,83.405,80.440,86.385,85.280,82.690,76.280,72.990,...,86.140,88.790,86.960,71.890,85.160,87.645,80.175,62.745,56.920,85.470
351,Hull and York Medical School,83.010,86.460,92.515,78.320,80.010,80.805,93.295,62.615,62.110,...,78.320,91.735,91.980,79.100,91.225,84.935,54.185,41.100,49.440,76.725


In [22]:
pd.read_pickle("data/unis_2122.pkl")

,University,Q01,Q02,Q03,Q04,Q05,Q06,Q07,Q08,Q09,...,Q18,Q19,Q20,Q21,Q22,Q23,Q24,Q25,Q26,Q27
0,University of Aberdeen,89.710,83.805,87.465,80.030,83.305,84.815,76.645,68.720,73.830,...,80.765,83.885,84.755,65.650,78.815,90.235,73.165,53.870,52.595,85.085
1,Abertay University,89.885,82.375,82.535,77.345,82.740,81.375,78.440,74.410,72.840,...,78.040,77.545,79.580,55.545,77.540,84.605,69.930,52.170,45.015,80.000
2,Aberystwyth University,91.130,85.960,87.490,79.215,84.445,85.285,80.410,79.010,78.460,...,83.050,82.975,83.150,66.420,79.960,89.240,79.970,68.370,61.290,85.165
3,Abingdon and Witney College,84.135,83.975,75.960,72.115,79.710,86.060,83.560,66.500,77.210,...,72.885,71.155,73.890,65.865,81.060,82.980,86.500,65.415,54.420,84.500
4,ACM Guildford Limited,72.950,66.890,56.565,56.515,59.605,65.260,59.180,58.430,57.515,...,50.790,50.445,52.125,43.700,61.310,49.255,55.705,29.470,28.670,44.585
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
348,University of York,87.670,82.130,86.295,74.125,83.050,83.740,73.430,66.965,69.055,...,80.180,83.165,83.420,59.365,75.970,84.455,67.900,44.450,41.100,78.975
349,York College,84.225,88.335,83.150,86.485,88.150,86.295,84.445,66.475,71.650,...,75.715,85.930,76.280,82.960,70.555,84.225,75.930,70.555,71.130,82.595
350,York St John University,86.610,81.475,82.565,78.455,81.960,82.590,76.970,73.375,73.825,...,75.420,80.700,78.585,65.885,78.710,82.640,73.895,54.250,56.750,77.995
351,Hull and York Medical School,77.170,82.070,92.415,77.335,69.335,80.280,93.980,57.075,63.960,...,73.915,85.540,89.470,70.660,88.980,79.655,55.170,42.885,36.350,73.625


In [23]:
pd.read_pickle("data/course_1819.pkl")

,course,Q01,Q02,Q03,Q04,Q05,Q06,Q07,Q08,Q09,...,Q18,Q19,Q20,Q21,Q22,Q23,Q24,Q25,Q26,Q27
0,"Agriculture, food and related studies",0.900618,0.843821,0.824274,0.792376,0.818120,0.845352,0.780188,0.758857,0.766305,...,0.788127,0.828712,0.838209,0.708674,0.859552,0.827324,0.771795,0.607940,0.521735,0.806877
1,"Architecture, building and planning",0.864557,0.819031,0.844623,0.813449,0.820257,0.861415,0.809947,0.699209,0.723022,...,0.773864,0.851451,0.838232,0.725760,0.877143,0.821880,0.755535,0.612328,0.561311,0.800795
2,Biological and sport sciences,0.905481,0.838039,0.850868,0.820695,0.841641,0.858782,0.818706,0.756396,0.760547,...,0.803015,0.826895,0.850702,0.723915,0.877515,0.847168,0.791181,0.648481,0.594250,0.841892
3,Business and management,0.881794,0.807948,0.807032,0.807564,0.827849,0.864841,0.806209,0.769411,0.770618,...,0.799264,0.831381,0.834810,0.699124,0.854965,0.832726,0.770047,0.645634,0.595923,0.820707
4,Combined and general studies,0.920170,0.844361,0.894328,0.795198,0.857059,0.869382,0.746130,0.748594,0.772340,...,0.846587,0.885151,0.883379,0.637948,0.754218,0.839859,0.777075,0.571040,0.560589,0.885969
5,Computing,0.847867,0.772430,0.814808,0.783982,0.782702,0.789667,0.817607,0.710932,0.773641,...,0.789454,0.738801,0.841701,0.686662,0.859334,0.830431,0.745298,0.594539,0.534316,0.779747
6,"Design, and creative and performing arts",0.879196,0.856107,0.814499,0.811817,0.843015,0.840946,0.841138,0.722946,0.744560,...,0.737508,0.785235,0.800031,0.748786,0.838651,0.815433,0.769659,0.631202,0.563142,0.792692
7,Education and teaching,0.905095,0.860798,0.861861,0.864664,0.895334,0.903583,0.887141,0.801408,0.794591,...,0.771876,0.827457,0.811271,0.733017,0.882785,0.855484,0.815892,0.652734,0.582293,0.855950
8,Engineering and technology,0.816795,0.737034,0.809306,0.780600,0.746390,0.782853,0.742191,0.672049,0.729158,...,0.781358,0.779565,0.811700,0.661250,0.856361,0.793829,0.704157,0.557417,0.499375,0.751268
9,"Geography, earth and environmental studies",0.920252,0.869336,0.869638,0.798402,0.882946,0.886400,0.809572,0.742481,0.748851,...,0.854315,0.873864,0.896543,0.733697,0.885046,0.873247,0.788024,0.596336,0.509598,0.871119


In [24]:
pd.read_pickle("data/course_2122.pkl")

,course,Q01,Q02,Q03,Q04,Q05,Q06,Q07,Q08,Q09,...,Q18,Q19,Q20,Q21,Q22,Q23,Q24,Q25,Q26,Q27
0,"Agriculture, food and related studies",0.848417,0.796876,0.780800,0.727643,0.760136,0.814108,0.712944,0.708093,0.714422,...,0.692980,0.733963,0.733107,0.614613,0.761544,0.787482,0.704984,0.522073,0.508743,0.733192
1,"Architecture, building and planning",0.813885,0.784609,0.800091,0.787250,0.799424,0.823585,0.770105,0.666583,0.680618,...,0.687076,0.786664,0.733472,0.641516,0.795885,0.773041,0.692967,0.528990,0.503756,0.747282
2,Biological and sport sciences,0.882982,0.821448,0.828843,0.774331,0.804320,0.829850,0.758726,0.734825,0.730646,...,0.745247,0.768711,0.793037,0.646765,0.799785,0.825923,0.751016,0.593922,0.558813,0.797755
3,Business and management,0.859937,0.781681,0.794805,0.789777,0.813643,0.852627,0.789272,0.754530,0.742113,...,0.771117,0.786547,0.794266,0.662147,0.811216,0.818649,0.747880,0.613163,0.606975,0.791721
4,Combined and general studies,0.854085,0.801465,0.838083,0.746840,0.813698,0.867948,0.713996,0.676573,0.673590,...,0.671960,0.775144,0.750127,0.531346,0.726315,0.769569,0.677802,0.454758,0.421910,0.747802
5,Computing,0.800518,0.723924,0.780689,0.742703,0.749784,0.757343,0.788717,0.682236,0.729056,...,0.737940,0.707409,0.788719,0.586694,0.784788,0.802527,0.686662,0.534956,0.508870,0.716932
6,"Design, and creative and performing arts",0.855815,0.835647,0.782713,0.777793,0.809103,0.817943,0.798044,0.703062,0.739555,...,0.676552,0.725123,0.724992,0.698280,0.780264,0.772424,0.722327,0.561054,0.527303,0.740642
7,Education and teaching,0.877508,0.825720,0.829790,0.816609,0.863886,0.877170,0.850110,0.789272,0.764035,...,0.737844,0.790709,0.750369,0.685350,0.816428,0.834714,0.776168,0.618973,0.585729,0.812984
8,Engineering and technology,0.813698,0.724424,0.807586,0.767849,0.750056,0.783391,0.741382,0.668698,0.720999,...,0.741788,0.752464,0.772206,0.624247,0.807494,0.796256,0.685466,0.526247,0.498772,0.741221
9,"Geography, earth and environmental studies",0.899859,0.845210,0.845088,0.765002,0.850873,0.871155,0.753549,0.731001,0.729653,...,0.759879,0.816272,0.797107,0.651979,0.797279,0.844992,0.741521,0.528089,0.513458,0.807472


In [25]:
pd.read_pickle("data/course_and_uni_1819.pkl")

,uni,course,Q01,Q02,Q03,Q04,Q05,Q06,Q07,Q08,...,Q18,Q19,Q20,Q21,Q22,Q23,Q24,Q25,Q26,Q27
0,ACM Guildford Limited,"Design, and creative and performing arts",0.807250,0.68250,0.51025,0.50355,0.651650,0.658200,0.654250,0.52195,...,0.612150,0.53395,0.643250,0.59545,0.75470,0.65185,0.617850,0.429400,0.368450,0.50085
1,AECC University College,Subjects allied to medicine,0.944450,0.90835,1.00000,0.83890,0.811100,0.858350,0.972200,0.66945,...,0.888900,0.98610,0.986100,0.91665,0.97220,0.82500,0.671450,0.444450,0.830550,0.87500
2,ALRA,"Design, and creative and performing arts",0.928550,0.92955,0.95275,0.96110,0.911000,0.939750,0.942550,0.65515,...,0.527250,0.65605,0.859050,0.94995,0.96110,0.92215,0.880350,0.880350,0.815900,0.92120
3,Aberystwyth University,"Agriculture, food and related studies",0.980925,0.88655,0.87435,0.82560,0.858525,0.915825,0.826375,0.84610,...,0.939425,0.90250,0.909075,0.74730,0.85045,0.86130,0.748725,0.685475,0.556975,0.92330
4,Aberystwyth University,Biological and sport sciences,0.956200,0.91390,0.92010,0.85765,0.931550,0.912400,0.899300,0.82725,...,0.937450,0.91200,0.924250,0.74590,0.95465,0.93755,0.873950,0.790100,0.580100,0.91370
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1493,University of the Highlands and Islands,"Historical, philosophical and religious studies",0.947150,0.89300,0.90020,0.78180,0.876150,0.885600,0.711200,0.78450,...,0.770800,0.76715,0.791650,0.50600,0.56215,0.89870,0.767650,0.600800,0.345350,0.90125
1494,University of the Highlands and Islands,Language and area studies,1.000000,1.00000,1.00000,1.00000,0.988100,1.000000,0.913950,0.94050,...,0.913950,1.00000,0.975000,0.82310,0.75910,0.97620,0.964300,0.842500,0.543200,0.96430
1495,University of the Highlands and Islands,Psychology,0.896850,0.91665,0.89685,0.89285,0.924600,0.892850,0.816650,0.86905,...,0.920650,0.84920,0.849200,0.61505,0.60555,0.97220,0.797600,0.619050,0.555600,0.89285
1496,University of the Highlands and Islands,Social sciences,0.855250,0.85455,0.90405,0.83660,0.906550,0.874050,0.746350,0.82460,...,0.784750,0.77125,0.744000,0.52205,0.58090,0.85455,0.741700,0.481300,0.398550,0.85170


In [26]:
pd.read_pickle("data/course_and_uni_2122.pkl")

,uni,course,Q01,Q02,Q03,Q04,Q05,Q06,Q07,Q08,...,Q18,Q19,Q20,Q21,Q22,Q23,Q24,Q25,Q26,Q27
0,ACM Guildford Limited,"Design, and creative and performing arts",0.72950,0.668900,0.565650,0.565150,0.59605,0.652600,0.59180,0.58430,...,0.50790,0.50445,0.521250,0.43700,0.613100,0.49255,0.557050,0.29470,0.286700,0.44585
1,AECC University College,Subjects allied to medicine,0.71025,0.645650,0.832950,0.664650,0.53340,0.669700,0.74945,0.51245,...,0.70740,0.81675,0.676500,0.67485,0.857500,0.62305,0.324250,0.25405,0.500900,0.48010
2,Abertay University,"Agriculture, food and related studies",0.97405,0.921700,0.934850,0.884200,0.92305,0.883450,0.88355,0.88485,...,0.87235,0.92105,0.907050,0.75660,0.922350,0.93555,0.858550,0.82040,0.653550,0.89670
3,Abertay University,Biological and sport sciences,0.92635,0.887000,0.858350,0.818400,0.86395,0.828150,0.77495,0.82315,...,0.82210,0.79380,0.813400,0.60250,0.723500,0.87595,0.813150,0.61340,0.507050,0.84730
4,Abertay University,Business and management,0.86455,0.739600,0.724100,0.803850,0.76565,0.833000,0.74735,0.71620,...,0.83470,0.80735,0.821450,0.56400,0.847150,0.81870,0.646450,0.49450,0.468450,0.78885
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1538,University of the Highlands and Islands,Language and area studies,0.94600,0.914750,0.968750,0.883500,0.92330,0.931800,0.94600,0.80115,...,0.83480,0.88790,0.825900,0.59820,0.589450,0.89730,0.889900,0.50300,0.511900,0.92115
1539,University of the Highlands and Islands,Medicine and dentistry,0.87065,0.877650,0.954550,0.954550,0.95455,0.954550,0.91610,0.66435,...,0.86365,0.64335,0.825200,0.80070,0.954550,0.95455,0.793700,0.54895,0.494950,0.91610
1540,University of the Highlands and Islands,Psychology,0.84245,0.809100,0.954550,0.775750,0.87575,0.875750,0.76365,0.73030,...,0.83335,0.87575,0.832200,0.61820,0.592850,0.85455,0.763650,0.52725,0.651550,0.80910
1541,University of the Highlands and Islands,Social sciences,0.87100,0.834000,0.867000,0.779650,0.89800,0.867000,0.76905,0.76945,...,0.76015,0.77585,0.741400,0.48065,0.487150,0.86550,0.719750,0.50315,0.476300,0.80950
